## Analyse Exploratoire

### Distribution
![Distribution](imgs/Distribution.png)

In [6]:
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import matplotlib

matplotlib.use("Qt5Agg")

# Charger le CSV
df = pd.read_csv("data/fichier-de-donnees-numeriques.csv")

# Description du fichier
print(f"Description : {df.describe}")

# Affichage de la distribution des données (les données sont toutes numériques)
plt.figure(figsize=(15, 12))

for i, col in enumerate(df.columns, 1):
    plt.subplot(3, 3, i)     # 3x3 pour 9 colonnes
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f"Distribution de {col}")
    plt.xlabel(col)
    plt.ylabel("Fréquence")

plt.tight_layout()
plt.show()

Description : <bound method NDFrame.describe of       age  taille  poids  revenu_estime_mois  historique_credits  \
0      73   161.1   67.3                 857                 NaN   
1      44   168.2   74.9                5245                 2.0   
2      71   160.3   45.5                3792                 0.0   
3      62   161.9   87.7                3291                 NaN   
4      18   178.0   77.6                3893                 NaN   
...   ...     ...    ...                 ...                 ...   
9995   37   163.6   96.7                3894                 1.0   
9996   19   157.2   55.3                2969                 0.0   
9997   71   163.0   77.2                2893                 NaN   
9998   54   165.7   73.2                1193                 NaN   
9999   54   173.9   77.8                 500                 NaN   

      risque_personnel  score_credit  loyer_mensuel  montant_pret  
0                 0.11         615.0        1377.97  13157.101646  

## Heatmap

![Correlation](imgs/Correlation.png)

In [3]:
# Heatmap de corrélation
plt.figure(figsize=(10, 8))
corr = df.corr(numeric_only=True)

sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matrice de corrélation")
plt.show()

## Identifier les valeurs manquantes

Utilisation de missingno

![Colonnes manquantes](imgs/Colonnes.png)
![Colonnes manquantes 2](imgs/Colonnes2.png)

In [5]:
# Visualiser la matrice des valeurs manquantes
msno.matrix(df)
plt.show()

# Visualiser le barplot des colonnes manquantes
msno.bar(df)
plt.show()

# Nombre de lignes avec au moins une valeur manquante
missing_line = df.isna().any(axis=1).sum()
total_line = df.shape[0]
print(f"Nb lignes manquantes : {missing_line} / {total_line}")

Nb lignes manquantes : 8446 / 10000


## Nettoyage des Données

### Identifier les colonnes utiles au résultat




### Supprimer les colonnes non pertinentes

3 colonnes sont partielles, les autres sont complètes. Je fais donc le choix d'éliminer des colonnes plutôt que les lignes avec des données manquantes.
Car si on éliminait les lignes incomplètes, on retirerait 8446 / 10000 lignes.

Cependant, pour rester compatible avec de futures données, je supprime sur condition de pourcentage de données manquantes. Car s'il manque seulement 1 données dans une colonnes on ne va pas supprimer la colonne pour autant !


In [9]:
cols_before = df.shape[1]
threshold = 0.20  # 20%
df_clean = df.drop(columns=df.columns[df.isna().mean() > threshold])
cols_after = df_clean.shape[1]
print(f"Nb colonnes avant nettoyage : {cols_before}, nb colonnes après : {cols_after}")


Nb colonnes avant nettoyage : 9, nb colonnes après : 6


Au cas où il resterait des lignes avec des champs manquants, je supprime les lignes concernée. Il n'y en a pas actuellement dans le jeu de données, mais cela pourrait être le cas dans le futur.

In [13]:
lines_before = df_clean.shape[0]
# Supprimer les lignes contenant des valeurs manquantes
df_clean = df_clean.dropna()
lines_after = df_clean.shape[0]
print(f"Nb lignes avant nettoyage : {lines_before}, nb lignes après : {lines_after}")

Nb lignes avant nettoyage : 10000, nb lignes après : 10000


### Ecarter les outliers
Filtrer les outliers avec la méthode IQR (sur toutes les colonnes)

In [15]:
lines_before = df_clean.shape[0]
Q1 = df_clean.quantile(0.25)
Q3 = df_clean.quantile(0.75)
IQR = Q3 - Q1

df_filtered = df_clean[~((df_clean < (Q1 - 1.5 * IQR)) | (df_clean > (Q3 + 1.5 * IQR))).any(axis=1)]

lines_after = df_filtered.shape[0]
print(f"Nb lignes avant filtrage des outliers : {lines_before}, nb lignes après : {lines_after}")


Nb lignes avant filtrage des outliers : 10000, nb lignes après : 9751


### Imputation
Pas d'imputation dans ce cas, car une fois les colonnes quasi-vides enlevées, il n'y a pas de ligne avec des valeurs nulles. 

### Catégorisation / Normalisation

Toutes les colonnes sont de types numériques, donc pas besoin des catégoriser. Il faudra par contre les normaliser.

#### Standardisation des données
La standardisation consiste à transformer chaque variable pour qu’elle ait une moyenne de 0 et un écart-type de 1.  
Cela permet de mettre toutes les colonnes numériques sur la même échelle, ce qui est essentiel pour de nombreux algorithmes de Machine Learning.




In [18]:
from sklearn.preprocessing import StandardScaler

# Initialiser le scaler
scaler = StandardScaler()

# Colonnes numériques (toutes celles de df_filtered)
num_cols = df_filtered.select_dtypes(include="number").columns

# Copie du dataset
df_standardized = df_filtered.copy()

# Application de la standardisation
df_standardized[num_cols] = scaler.fit_transform(df_filtered[num_cols])

print("Standardisation effectuée (aperçu) :")
display(df_standardized.head())

# Sauvegarde du dataset standardisé
output_path = "data/dataset_standardized.csv"
df_standardized.to_csv(output_path, index=False)

print(f"Dataset sauvegardé sous : {output_path}")



Standardisation effectuée (aperçu) :
Dataset sauvegardé sous : data/dataset_standardized.csv
